In [1]:
from neo4j import GraphDatabase


In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


In [4]:
conn = Neo4jConnection(uri="bolt://neo4j:7687", user="neo4j", pwd="uom-pkg-1")


Failed to create the driver: Cannot resolve address neo4j:7687


In [3]:
#conn = Neo4jConnection(uri="bolt://192.168.0.203:7687", user="neo4j", pwd="uom-pkg-1")

conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="uom-pkg-1")


In [3]:
conn = Neo4jConnection(uri="neo4j+s://c3d71d16.databases.neo4j.io", user="neo4j", pwd="14QXSBobvY_nnVOUBZN1bUAmW8Tz300P08hOgzOKdNg")


In [7]:
conn.query('CREATE CONSTRAINT q_value IF NOT EXISTS ON (n:Node) ASSERT n.id IS UNIQUE')

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid constraint syntax, ON and ASSERT should not be used. Replace ON with FOR and ASSERT with REQUIRE. (line 1, column 41 (offset: 40))
"CREATE CONSTRAINT q_value IF NOT EXISTS ON (n:Node) ASSERT n.id IS UNIQUE"
                                         ^}


In [4]:
conn.query('CREATE CONSTRAINT q_value IF NOT EXISTS FOR (n:Node) REQUIRE n.id IS UNIQUE')

[]

In [5]:
cqlNodeQuery          = "MATCH (x:university) RETURN x"

cqlCreate = """CREATE (cornell:university { name: "Cornell University"}),
(yale:university { name: "Yale University"}),
(princeton:university { name: "Princeton University"}),
(harvard:university { name: "Harvard University"}),

(cornell)-[:connects_in {miles: 259}]->(yale),
(cornell)-[:connects_in {miles: 210}]->(princeton),
(cornell)-[:connects_in {miles: 327}]->(harvard),

(yale)-[:connects_in {miles: 259}]->(cornell),
(yale)-[:connects_in {miles: 133}]->(princeton),
(yale)-[:connects_in {miles: 133}]->(harvard),

(harvard)-[:connects_in {miles: 327}]->(cornell),
(harvard)-[:connects_in {miles: 133}]->(yale),
(harvard)-[:connects_in {miles: 260}]->(princeton),

(princeton)-[:connects_in {miles: 210}]->(cornell),
(princeton)-[:connects_in {miles: 133}]->(yale),
(princeton)-[:connects_in {miles: 260}]->(harvard)"""

conn.query(cqlCreate)
nodes = conn.query(cqlNodeQuery)
for node in nodes:
    print(node)


<Record x=<Node id=0 labels=frozenset({'university'}) properties={'name': 'Cornell University'}>>
<Record x=<Node id=1 labels=frozenset({'university'}) properties={'name': 'Yale University'}>>
<Record x=<Node id=2 labels=frozenset({'university'}) properties={'name': 'Princeton University'}>>
<Record x=<Node id=3 labels=frozenset({'university'}) properties={'name': 'Harvard University'}>>


In [18]:
cqlCreate2 = """
CREATE (VariableAssignment:KnowledgeUnit {externalId: 1,
name: "VariableAssignment",
description: "You can assign a value to a variable to use that value in different operations.", 
code_example: "x = 23
size = 120
name = ’Joe’", 
exercise: "Create a variable, name it to max_speed and assign 130 to it."}),

(Literals:KnowledgeUnit { externalId: 2,
name: "Literals",
description: "Quantities/ notations whose value does not change during the execution of a program. Also known as constants. 
String Literals
Numeric Literals
Boolean Literals
Literal Collections
Special Literals", 
code_example: "423
-87
True", 
exercise: "Use numeric literals to calculate 5 multiplied by 8."}),

(VariableAssignment)-[:uses]->(Literals)

"""
conn.query(cqlCreate2)


[]

In [8]:
cqlMatch = """
MATCH (n) RETURN n
"""
conn.query(cqlMatch)


Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'commit': expected
  "ALTER"
  "CALL"
  "CREATE"
  "DEALLOCATE"
  "DELETE"
  "DENY"
  "DETACH"
  "DROP"
  "DRYRUN"
  "ENABLE"
  "FOREACH"
  "GRANT"
  "LOAD"
  "MATCH"
  "MERGE"
  "OPTIONAL"
  "REALLOCATE"
  "REMOVE"
  "RENAME"
  "RETURN"
  "REVOKE"
  "SET"
  "SHOW"
  "START"
  "STOP"
  "TERMINATE"
  "UNWIND"
  "USE"
  "USING"
  "WITH" (line 2, column 1 (offset: 1))
"commit"
 ^}


In [5]:
cqlCreate3 = """
CREATE (Introduction:KnowledgeUnit { externalId: "Introduction",
name: "Introduction",
description: "It needs to be started somewhere: Python is a programming language. It is named after the BBC show Monty Python’s Flying Circus",
timeInstances: [],
evokers: []}),

(Literal:KnowledgeUnit { externalId: "Literal",
name: "Literal",
description: "Quantities/ notations whose value does not change during the execution of a program. Also known as constants. 
String Literals
Numeric Literals
Boolean Literals
Literal Collections
Special Literals",
timeInstances: [],
evokers: []}),

(WriteToConsole:KnowledgeUnit { externalId: "WriteToConsole",
name: "Write to the console",
description: "Print the specified message to the screen, or other standard output device. The message can be a string, or any other object, the object will be converted into a string before written to the screen.",
timeInstances: [],
evokers: ["print"]}),

(DetermineObjectType:KnowledgeUnit { externalId: "DetermineObjectType",
name: "Determine object type",
description: "The type of a Python object determines what kind of object it is; every object has a type. An object’s type is accessible as its __class__ attribute or can be retrieved with type(obj).",
timeInstances: [],
evokers: ["type"]}),

(ListDataType:KnowledgeUnit { externalId: "ListDataType",
name: "List data type",
description: "Lists are iterable objects. How to set a list element, how to delete a list element and how to get a list element.",
timeInstances: [],
evokers: ["[", "]"]}),

(ForLoop:KnowledgeUnit { externalId: "ForLoop",
name: "For loop",
description: "It iterates over an iterable object, capturing each element to a local variable for use by the attached block.",
timeInstances: [],
evokers: ["for", "in"]}),

(CodeBlock:KnowledgeUnit { externalId: "CodeBlock",
name: "Code block",
description: "Program code that belongs together. Python uses whitespace indentation, rather than curly brackets or keywords, to delimit blocks. An increase in indentation comes after certain statements; a decrease in indentation signifies the end of the current block. Thus, the program's visual structure accurately represents its semantic structure.",
timeInstances: [],
evokers: []}),

(AssignmentStatement:KnowledgeUnit { externalId: "AssignmentStatement",
name: "Assignment statement",
description: "The assignment statement (=) binds a name as a reference to a separate, dynamically allocated object. Variables may subsequently be rebound at any time to any object.",
timeInstances: [],
evokers: ["="]}),

(Statement:KnowledgeUnit { externalId: "Statement",
name: "Statement",
description: "In computer programming, a statement is a syntactic unit of an imperative programming language that expresses some action to be carried out. A program written in such a language is formed by a sequence of one or more statements. A statement may have internal components (e.g. expressions).",
timeInstances: [],
evokers: []}),

(StringLiteral:KnowledgeUnit { externalId: "StringLiteral",
name: "String literal",
description: "A string literal or anonymous string is a literal for a string value in the source code of a computer program. Modern programming languages commonly use a quoted sequence of characters, formally 'bracketed delimiters', as in x = 'foo', where 'foo' is a string literal with value foo. ",
timeInstances: [],
evokers: []}),

(StringDataType:KnowledgeUnit { externalId: "StringDataType",
name: "String data type",
description: "String variables - which are also called alphanumeric variables or character variables -- have values that are treated as text.",
timeInstances: [],
evokers: []}),

(Variable:KnowledgeUnit { externalId: "Variable",
name: "Variable",
description: "Value storage",
timeInstances: [],
evokers: ["="]}),

(WhileLoop:KnowledgeUnit { externalId: "WhileLoop",
name: "While loop",
description: "Executes a block of code as long as its condition is true.",
timeInstances: [],
evokers: ["while"]}),

(Condition:KnowledgeUnit { externalId: "Condition",
name: "Condition",
description: "Conditionally executes a block of code.",
timeInstances: [],
evokers: ["if"]}),

(ExceptionHandling:KnowledgeUnit { externalId: "ExceptionHandling",
name: "Exception handling",
description: "Allows exceptions raised in its attached code block to be caught and handled.",
timeInstances: [],
evokers: ["try:", "except"]}),

(RaiseException:KnowledgeUnit { externalId: "RaiseException",
name: "Raise exception",
description: "Used to raise a specified exception or re-raise a caught exception.",
timeInstances: [],
evokers: ["raise"]}),

(Exception:KnowledgeUnit { externalId: "Exception",
name: "Exception",
description: "Unexpected events which will disrupt the normal flow of execution of instructions.",
timeInstances: [],
evokers: ["exception", "error"]}),

(Function:KnowledgeUnit { externalId: "Function",
name: "Function",
description: "A function or subroutine is a sequence of program instructions that performs a specific task, packaged as a unit. This unit can then be used in programs wherever that particular task should be performed.",
timeInstances: [],
evokers: ["def", "return"]}),

(ResourceManagement:KnowledgeUnit { externalId: "ResourceManagement",
name: "Resource management",
description: "Enclosing a code block within a context manager (for example, acquiring a lock before it is run, then releasing the lock; or opening and closing a file).",
timeInstances: [],
evokers: ["with", "open"]}),

(ModuleImport:KnowledgeUnit { externalId: "ModuleImport",
name: "Module import",
description: "Used to import modules whose functions or variables can be used in the current program.",
timeInstances: [],
evokers: ["import"]}),

(VariableNaming:KnowledgeUnit { externalId: "VariableNaming",
name: "Variable naming",
description: "Use  long speaking names for variables. Shorter temporary names can be used in longer formulas to achieve clean and understandable code at the same time.",
timeInstances: [],
evokers: ["="]}),

(VariableAssignment:KnowledgeUnit { externalId: "VariableAssignment",
name: "Variable assignment",
description: "You can assign a value to a variable to use that value in different operations.",
timeInstances: ["3.8", "3.9", "3.10"],
evokers: ["="]}),

(Algorithm:KnowledgeUnit { externalId: "Algorithm",
name: "Algorithm",
description: "A list set of instructions, used to solve problems or perform tasks, based on the understanding of available alternatives.",
timeInstances: [],
evokers: []}),

(ModifyACollectionWithCopy:KnowledgeUnit { externalId: "ModifyACollectionWithCopy",
name: "Modify a collection with copy",
description: "Strategy1:  Iterate over a copy; Strategy2:  Create a new collection",
timeInstances: [],
evokers: ["for", "in", "=="]}),

(ErrorHandling:KnowledgeUnit { externalId: "ErrorHandling",
name: "Error handling",
description: "The process of responding to and recovering from error conditions in your program.",
timeInstances: [],
evokers: ["try", "except", "finally"]}),

(PythonLanguageBasics:KnowledgeUnit { externalId: "PythonLanguageBasics",
name: "Python language basics",
description: "Basic level of programming in Python.",
timeInstances: [],
evokers: []}),

(DataType:KnowledgeUnit { externalId: "DataType",
name: "Data type",
description: "Data types in general",
timeInstances: [],
evokers: []}),

(RemovePrefixSuffixFunction:KnowledgeUnit { externalId: "RemovePrefixSuffixFunction",
name: "Remove string prefix and suffix function",
description: "A string has a beginning and an ending part that can be considered special. These are called the prefix and suffix. Removing specified text from prefix or suffix is possible by calling a certain function.",
timeInstances: ["3.9", "3.10", "3.11", "3.12"],
evokers: ["prefix", "suffix", "removeprefix(", "removesuffix("]}),

(PrefixSuffixHandling:KnowledgeUnit { externalId: "PrefixSuffixHandling",
name: "String prefix and suffix handling",
description: "A string has a beginning and an ending part that can be considered special. These are called the prefix and suffix. To remove specified text from prefix or suffix you need to implement your own solution.",
timeInstances: ["3.6", "3.7", "3.8"],
evokers: ["prefix", "suffix", "startswith", "endswith"]}),

(NumberOfItems:KnowledgeUnit { externalId: "NumberOfItems",
name: "Determine number of items",
description: "Return the length (the number of items) of an object. The argument may be a sequence (such as a string, bytes, tuple, list, or range) or a collection (such as a dictionary, set, or frozen set).",
timeInstances: [],
evokers: ["len("]}),

(BuiltInFunction:KnowledgeUnit { externalId: "BuiltInFunction",
name: "Built in function",
description: "The Python interpreter has a number of functions and types built into it that are always available.",
timeInstances: [],
evokers: []}),




(Literal)-[:prerequisite_of]->(VariableAssignment),
(AssignmentStatement)-[:prerequisite_of]->(VariableAssignment),
(VariableAssignment)-[:prerequisite_of]->(ListDataType),
(ListDataType)-[:prerequisite_of]->(ForLoop),
(VariableAssignment)-[:prerequisite_of]->(ForLoop),
(CodeBlock)-[:prerequisite_of]->(ForLoop),
(CodeBlock)-[:prerequisite_of]->(WhileLoop),
(Condition)-[:prerequisite_of]->(WhileLoop),
(CodeBlock)-[:prerequisite_of]->(Condition),
(StringLiteral)-[:prerequisite_of]->(StringDataType),
(StringDataType)-[:prerequisite_of]->(WriteToConsole),
(VariableAssignment)-[:prerequisite_of]->(DetermineObjectType),
(Variable)-[:prerequisite_of]->(VariableNaming),
(VariableAssignment)-[:prerequisite_of]->(VariableNaming),

(ForLoop)-[:prerequisite_of]->(ModifyACollectionWithCopy),
(Condition)-[:prerequisite_of]->(ModifyACollectionWithCopy),
(ListDataType)-[:prerequisite_of]->(ModifyACollectionWithCopy),

(ErrorHandling)-[:prerequisite_of]->(Algorithm),
(StringDataType)-[:prerequisite_of]->(RemovePrefixSuffixFunction),
(StringDataType)-[:prerequisite_of]->(PrefixSuffixHandling),
(Condition)-[:prerequisite_of]->(PrefixSuffixHandling),
(NumberOfItems)-[:prerequisite_of]->(PrefixSuffixHandling),
(StringDataType)-[:prerequisite_of]->(NumberOfItems),
(BuiltInFunction)-[:prerequisite_of]->(Function),


(ForLoop)-[:is_a]->(Statement),
(WhileLoop)-[:is_a]->(Statement),
(AssignmentStatement)-[:is_a]->(Statement),
(StringLiteral)-[:is_a]->(Literal),
(ModifyACollectionWithCopy)-[:is_a]->(Algorithm),
(ExceptionHandling)-[:is_a]->(ErrorHandling),
(RaiseException)-[:is_a]->(ErrorHandling),
(Exception)-[:is_a]->(ErrorHandling),
(ErrorHandling)-[:is_a]->(PythonLanguageBasics),
(Function)-[:is_a]->(PythonLanguageBasics),
(Introduction)-[:is_a]->(PythonLanguageBasics),
(ModuleImport)-[:is_a]->(PythonLanguageBasics),
(Algorithm)-[:is_a]->(PythonLanguageBasics),
(Statement)-[:is_a]->(PythonLanguageBasics),
(CodeBlock)-[:is_a]->(PythonLanguageBasics),
(ResourceManagement)-[:is_a]->(PythonLanguageBasics),
(Literal)-[:is_a]->(PythonLanguageBasics),
(Variable)-[:is_a]->(PythonLanguageBasics),
(DataType)-[:is_a]->(PythonLanguageBasics),
(ListDataType)-[:is_a]->(DataType),
(StringDataType)-[:is_a]->(DataType),
(NumberOfItems)-[:is_a]->(Function),
(RemovePrefixSuffixFunction)-[:is_a]->(Function),
(BuiltInFunction)-[:is_a]->(PythonLanguageBasics),
(NumberOfItems)-[:is_a]->(BuiltInFunction),


(Test100:TestUnit { externalId: 100,
description: "One type of variable assignment test.",
url: "multiple-choice",
timeInstances: ["3.9"]}),

(Test101:TestUnit { externalId: 101,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test102:TestUnit { externalId: 102,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test103:TestUnit { externalId: 103,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test104:TestUnit { externalId: 104,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test105:TestUnit { externalId: 105,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test106:TestUnit { externalId: 106,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test107:TestUnit { externalId: 107,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test108:TestUnit { externalId: 108,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test109:TestUnit { externalId: 109,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test110:TestUnit { externalId: 110,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test111:TestUnit { externalId: 111,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test112:TestUnit { externalId: 112,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test113:TestUnit { externalId: 113,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test114:TestUnit { externalId: 114,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test115:TestUnit { externalId: 115,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test116:TestUnit { externalId: 116,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test117:TestUnit { externalId: 117,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test118:TestUnit { externalId: 118,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test119:TestUnit { externalId: 119,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test120:TestUnit { externalId: 120,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test121:TestUnit { externalId: 121,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test122:TestUnit { externalId: 122,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test123:TestUnit { externalId: 123,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test124:TestUnit { externalId: 124,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test125:TestUnit { externalId: 125,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test126:TestUnit { externalId: 126,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test127:TestUnit { externalId: 127,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.9"]}),

(Test128:TestUnit { externalId: 128,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test129:TestUnit { externalId: 129,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),

(Test130:TestUnit { externalId: 130,
description: "Another type of test",
url: "multiple-choice",
timeInstances: ["3.8"]}),



(Test100)-[:tests]->(VariableAssignment),
(Test101)-[:tests]->(VariableAssignment),
(Test102)-[:tests]->(Variable),
(Test103)-[:tests]->(Literal),
(Test104)-[:tests]->(WriteToConsole),
(Test105)-[:tests]->(DetermineObjectType),
(Test106)-[:tests]->(ListDataType),
(Test107)-[:tests]->(ForLoop),
(Test108)-[:tests]->(CodeBlock),
(Test109)-[:tests]->(AssignmentStatement),
(Test111)-[:tests]->(Statement),
(Test112)-[:tests]->(StringLiteral),
(Test113)-[:tests]->(StringDataType),
(Test114)-[:tests]->(Variable),
(Test115)-[:tests]->(WhileLoop),
(Test116)-[:tests]->(Condition),
(Test117)-[:tests]->(ExceptionHandling),
(Test118)-[:tests]->(RaiseException),
(Test119)-[:tests]->(Exception),
(Test120)-[:tests]->(Function),
(Test121)-[:tests]->(ResourceManagement),
(Test122)-[:tests]->(ModuleImport),
(Test123)-[:tests]->(VariableNaming),
(Test124)-[:tests]->(ErrorHandling),
(Test125)-[:tests]->(Algorithm),
(Test126)-[:tests]->(ModifyACollectionWithCopy),
(Test127)-[:tests]->(RemovePrefixSuffixFunction),
(Test128)-[:tests]->(PrefixSuffixHandling),
(Test129)-[:tests]->(NumberOfItems),
(Test130)-[:tests]->(BuiltInFunction),

(Material1000:MaterialUnit { externalId: 1000,
description: "Some introduction to version 3.8.*",
url: "https://localhost/uom-python-material/1000",
timeInstances: ["3.8"]}),

(Material1001:MaterialUnit { externalId: 1001,
description: "Intro text to version 3.9.*",
url: "https://localhost/uom-python-material/1001",
timeInstances: ["3.9"]}),

(Material1002:MaterialUnit { externalId: 1002,
description: "Intro text to version 3.10.*",
url: "https://localhost/uom-python-material/1002",
timeInstances: ["3.10"]}),

(Material1003:MaterialUnit { externalId: 1003,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1003",
timeInstances: ["3.8"]}),

(Material1004:MaterialUnit { externalId: 1004,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1004",
timeInstances: ["3.8"]}),

(Material1005:MaterialUnit { externalId: 1005,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1005",
timeInstances: ["3.8"]}),

(Material1006:MaterialUnit { externalId: 1006,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1006",
timeInstances: ["3.8"]}),

(Material1007:MaterialUnit { externalId: 1007,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1007",
timeInstances: ["3.8"]}),

(Material1008:MaterialUnit { externalId: 1008,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1008",
timeInstances: ["3.8"]}),

(Material1009:MaterialUnit { externalId: 1009,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1009",
timeInstances: ["3.8"]}),

(Material1010:MaterialUnit { externalId: 1010,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1010",
timeInstances: ["3.8"]}),

(Material1011:MaterialUnit { externalId: 1011,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1011",
timeInstances: ["3.8"]}),

(Material1012:MaterialUnit { externalId: 1012,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1012",
timeInstances: ["3.8"]}),

(Material1013:MaterialUnit { externalId: 1013,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1013",
timeInstances: ["3.8"]}),

(Material1014:MaterialUnit { externalId: 1014,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1014",
timeInstances: ["3.8"]}),

(Material1015:MaterialUnit { externalId: 1015,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1015",
timeInstances: ["3.8"]}),

(Material1016:MaterialUnit { externalId: 1016,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1016",
timeInstances: ["3.8"]}),

(Material1017:MaterialUnit { externalId: 1017,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1017",
timeInstances: ["3.8"]}),

(Material1018:MaterialUnit { externalId: 1018,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1018",
timeInstances: ["3.8"]}),

(Material1019:MaterialUnit { externalId: 1019,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1019",
timeInstances: ["3.8"]}),

(Material1020:MaterialUnit { externalId: 1020,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1020",
timeInstances: ["3.8"]}),

(Material1021:MaterialUnit { externalId: 1021,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1021",
timeInstances: ["3.8"]}),

(Material1022:MaterialUnit { externalId: 1022,
description: "Material unit of the connected KN",
url: "1022.pdf",
timeInstances: ["3.8"]}),

(Material1023:MaterialUnit { externalId: 1023,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1023",
timeInstances: ["3.8"]}),

(Material1024:MaterialUnit { externalId: 1024,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1024",
timeInstances: ["3.8"]}),

(Material1025:MaterialUnit { externalId: 1025,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1025",
timeInstances: ["3.8"]}),

(Material1026:MaterialUnit { externalId: 1026,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1026",
timeInstances: ["3.8"]}),

(Material1027:MaterialUnit { externalId: 1027,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1027",
timeInstances: ["3.8"]}),

(Material1028:MaterialUnit { externalId: 1028,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1028",
timeInstances: ["3.9"]}),

(Material1029:MaterialUnit { externalId: 1029,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1029",
timeInstances: ["3.8"]}),

(Material1030:MaterialUnit { externalId: 1030,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1030",
timeInstances: ["3.8"]}),

(Material1031:MaterialUnit { externalId: 1031,
description: "Material unit of the connected KN",
url: "https://localhost/uom-python-material/1031",
timeInstances: ["3.8"]}),


(Material1000)-[:material_of]->(Introduction),
(Material1001)-[:material_of]->(Introduction),
(Material1002)-[:material_of]->(Introduction),
(Material1003)-[:material_of]->(Literal),
(Material1004)-[:material_of]->(WriteToConsole),
(Material1005)-[:material_of]->(DetermineObjectType),
(Material1006)-[:material_of]->(ListDataType),
(Material1007)-[:material_of]->(ForLoop),
(Material1008)-[:material_of]->(CodeBlock),
(Material1009)-[:material_of]->(AssignmentStatement),
(Material1011)-[:material_of]->(Statement),
(Material1012)-[:material_of]->(StringLiteral),
(Material1013)-[:material_of]->(StringDataType),
(Material1014)-[:material_of]->(Variable),
(Material1015)-[:material_of]->(WhileLoop),
(Material1016)-[:material_of]->(Condition),
(Material1017)-[:material_of]->(ExceptionHandling),
(Material1018)-[:material_of]->(RaiseException),
(Material1019)-[:material_of]->(Exception),
(Material1020)-[:material_of]->(Function),
(Material1021)-[:material_of]->(ResourceManagement),
(Material1022)-[:material_of]->(ModuleImport),
(Material1023)-[:material_of]->(VariableNaming),
(Material1024)-[:material_of]->(VariableAssignment),
(Material1025)-[:material_of]->(Algorithm),
(Material1026)-[:material_of]->(ModifyACollectionWithCopy),
(Material1027)-[:material_of]->(ErrorHandling),

(Material1028)-[:material_of]->(RemovePrefixSuffixFunction),
(Material1029)-[:material_of]->(PrefixSuffixHandling),
(Material1030)-[:material_of]->(NumberOfItems),
(Material1031)-[:material_of]->(BuiltInFunction)


"""
conn.query(cqlCreate3)


[]

In [6]:
fruits = ["Apple", "Banana", "Lemon", "Grapes" ]
for fruit in fruits:
    print(fruit)

Apple
Banana
Lemon
Grapes


In [6]:
cqlCreate4 = """

CREATE 
(Program_v1:KnowledgeUnit { externalId: "Program_v1",
name: "EKSG Program v1", 
description: "A program description to demonstrate the potential of EKSG model.", 
timeInstances: ["v1"], 
evokers: [],
topic: "EKSGDemo_1"}),

(Program_v2:KnowledgeUnit { externalId: "Program_v2", 
name: "EKSG Program v2", 
description: "A program description to demonstrate the potential of EKSG model.", 
timeInstances: ["v2"], 
evokers: [],
topic: "EKSGDemo_1"}), 

(Program_v3:KnowledgeUnit { externalId: "Program_v3", 
name: "EKSG Program v3", 
description: "A program description to demonstrate the potential of EKSG model.", 
timeInstances: ["v3"], 
evokers: [],
topic: "EKSGDemo_1"}),

(Program_v4:KnowledgeUnit { externalId: "Program_v4", 
name: "EKSG Program v4", 
description: "A program description to demonstrate the potential of EKSG model.", 
timeInstances: ["v4"], 
evokers: [],
topic: "EKSGDemo_1"}), 

(ServiceV1:KnowledgeUnit { externalId: "ServiceV1", 
name: "EKSG Service v1", 
description: "A service description.", 
timeInstances: ["v1", "v2", "v3"], 
evokers: [],
topic: "EKSGDemo_1"}), 
 
(ServiceV2:KnowledgeUnit { externalId: "ServiceV2", 
name: "EKSG Service v2", 
description: "A service description.", 
timeInstances: ["v2", "v3"], 
evokers: [],
topic: "EKSGDemo_1"}), 
 
(ServiceV3:KnowledgeUnit { externalId: "ServiceV3", 
name: "EKSG Service v3", 
description: "A service description.", 
timeInstances: ["v3"], 
evokers: [],
topic: "EKSGDemo_1"}), 
 
(ServiceV4:KnowledgeUnit { externalId: "ServiceV4", 
name: "EKSG Service v4", 
description: "A service description.", 
timeInstances: ["v4"], 
evokers: [],
topic: "EKSGDemo_1"}), 

(BasicFeature:KnowledgeUnit { externalId: "BasicFeature", 
name: "EKSG BasicFeature", 
description: "A BasicFeature description.", 
timeInstances: ["v1", "v2", "v3"], 
evokers: [],
topic: "EKSGDemo_1"}), 

(BetterFeature:KnowledgeUnit { externalId: "BetterFeature", 
name: "EKSG BetterFeature", 
description: "A BetterFeature description.", 
timeInstances: ["v3"], 
evokers: [],
topic: "EKSGDemo_1"}), 

(BestFeature:KnowledgeUnit { externalId: "BestFeature", 
name: "EKSG BestFeature", 
description: "A BestFeature description.", 
timeInstances: ["v4"], 
evokers: [],
topic: "EKSGDemo_1"}), 

(Feature:KnowledgeUnit { externalId: "Feature", 
name: "EKSG Feature", 
description: "The Feature description.", 
timeInstances: [], 
evokers: [],
topic: "EKSGDemo_1"}),

(Feature)-[:prerequisite_of]->(BasicFeature),
(Feature)-[:prerequisite_of]->(BetterFeature), 
(Feature)-[:prerequisite_of]->(BestFeature), 
(BasicFeature)-[:prerequisite_of]->(ServiceV2), 
(BasicFeature)-[:prerequisite_of]->(ServiceV1), 
(BetterFeature)-[:prerequisite_of]->(ServiceV3),
(BestFeature)-[:prerequisite_of]->(ServiceV4),
(ServiceV1)-[:prerequisite_of]->(Program_v1),
(ServiceV1)-[:prerequisite_of]->(Program_v2), 
(ServiceV1)-[:prerequisite_of]->(Program_v3), 
(ServiceV2)-[:prerequisite_of]->(Program_v2),
(ServiceV2)-[:prerequisite_of]->(Program_v3),
(ServiceV3)-[:prerequisite_of]->(Program_v3),
(ServiceV4)-[:prerequisite_of]->(Program_v4), 

(Material2000:MaterialUnit { externalId: 2000,
description: "Material unit",
url: "2000.pdf",
timeInstances: []}),

(Material2001:MaterialUnit { externalId: 2001,
description: "Material unit",
url: "2001.pdf",
timeInstances: []}),

(Material2002:MaterialUnit { externalId: 2002,
description: "Material unit",
url: "2002.pdf",
timeInstances: []}),

(Material2003:MaterialUnit { externalId: 2003,
description: "Material unit",
url: "2003.pdf",
timeInstances: []}),

(Material2004:MaterialUnit { externalId: 2004,
description: "Material unit",
url: "2004.pdf",
timeInstances: []}),

(Material2005:MaterialUnit { externalId: 2005,
description: "Material unit",
url: "2005.pdf",
timeInstances: []}),

(Material2006:MaterialUnit { externalId: 2006,
description: "Material unit",
url: "2006.pdf",
timeInstances: []}),

(Material2007:MaterialUnit { externalId: 2007,
description: "Material unit",
url: "2007.pdf",
timeInstances: []}),

(Material2008:MaterialUnit { externalId: 2008,
description: "Material unit",
url: "2008.pdf",
timeInstances: []}),

(Material2009:MaterialUnit { externalId: 2009,
description: "Material unit",
url: "2009.pdf",
timeInstances: []}),

(Material2010:MaterialUnit { externalId: 2010,
description: "Material unit",
url: "2010.pdf",
timeInstances: []}),

(Material2011:MaterialUnit { externalId: 2011,
description: "Material unit",
url: "2011.pdf",
timeInstances: []}),

(Material2000)-[:material_of]->(Program_v1),
(Material2001)-[:material_of]->(Program_v2),
(Material2002)-[:material_of]->(Program_v3),
(Material2003)-[:material_of]->(Program_v4),
(Material2004)-[:material_of]->(ServiceV1),
(Material2005)-[:material_of]->(ServiceV2),
(Material2006)-[:material_of]->(ServiceV3),
(Material2007)-[:material_of]->(ServiceV4),
(Material2008)-[:material_of]->(BasicFeature),
(Material2009)-[:material_of]->(BetterFeature),
(Material2010)-[:material_of]->(BestFeature),
(Material2011)-[:material_of]->(Feature),

(Test200:TestUnit { externalId: 200,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test201:TestUnit { externalId: 201,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test202:TestUnit { externalId: 202,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test203:TestUnit { externalId: 203,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test204:TestUnit { externalId: 204,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test205:TestUnit { externalId: 205,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test206:TestUnit { externalId: 206,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test207:TestUnit { externalId: 207,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test208:TestUnit { externalId: 208,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test209:TestUnit { externalId: 209,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test210:TestUnit { externalId: 210,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test211:TestUnit { externalId: 211,
description: "Another type of test",
url: "multiple-choice",
timeInstances: []}),

(Test200)-[:tests]->(Program_v1),
(Test201)-[:tests]->(Program_v2),
(Test202)-[:tests]->(Program_v3),
(Test203)-[:tests]->(Program_v4),
(Test204)-[:tests]->(ServiceV1),
(Test205)-[:tests]->(ServiceV2),
(Test206)-[:tests]->(ServiceV3),
(Test207)-[:tests]->(ServiceV4),
(Test208)-[:tests]->(BasicFeature),
(Test209)-[:tests]->(BetterFeature),
(Test210)-[:tests]->(BestFeature),
(Test211)-[:tests]->(Feature)

"""
conn.query(cqlCreate4)


[]

In [4]:
cqlCreate5 = """

CREATE 
(SqlApiJellemzese:KnowledgeUnit { externalId: "SqlApiJellemzese",
  name: "SQL API jellemzése", 
  description: "Általános SQL API és middleware koncepciók", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(JdbcAlapfogalmak:KnowledgeUnit { externalId: "JdbcAlapfogalmak", 
  name: "JDBC alapfogalmak", 
  description: "JDBC jellemzői, működési alapelvek, architektúra", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(JdbcDriverKezeles:KnowledgeUnit { externalId: "JdbcDriverKezeles", 
  name: "JDBC driver kezelése", 
  description: "JDBC driver típusok, telepítés, betöltés, regisztráció", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(JdbcOsztalyok:KnowledgeUnit { externalId: "JdbcOsztalyok", 
  name: "JDBC osztályok", 
  description: "JDBC főbb osztályai (Connection, Statement, stb.)", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(KapcsolatKiepitese:KnowledgeUnit { externalId: "KapcsolatKiepitese", 
  name: "Kapcsolat kiépítése", 
  description: "Adatbázis-kapcsolat létrehozása és konfigurálása", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(DmlParancsok:KnowledgeUnit { externalId: "DmlParancsok", 
  name: "DML parancsok", 
  description: "DML parancsok kiadása, végrehajtása, batch feldolgozás", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(DqlParancsok:KnowledgeUnit { externalId: "DqlParancsok", 
  name: "DQL parancsok", 
  description: "Lekérdezések végrehajtása, ResultSet kezelése", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(Hibakezeles:KnowledgeUnit { externalId: "Hibakezeles", 
  name: "Hibakezelés", 
  description: "JDBC hibakezelés (SQLException, SQLWarning)", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(PreparedStatementHasznalata:KnowledgeUnit { externalId: "PreparedStatementHasznalata", 
  name: "Prepared statement használata", 
  description: "Paraméterezett parancsok, biztonságos végrehajtás", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(TranzakcioKezeles:KnowledgeUnit { externalId: "TranzakcioKezeles", 
  name: "Tranzakciókezelés", 
  description: "Commit, rollback, autocommit", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(MetaadatKezeles:KnowledgeUnit { externalId: "MetaadatKezeles", 
  name: "Metaadat kezelés", 
  description: "DatabaseMetaData és ResultSetMetaData használata", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(CallableStatementHasznalata:KnowledgeUnit { externalId: "CallableStatementHasznalata", 
  name: "Callable statement használata", 
  description: "Tárolt eljárások/függvények hívása", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(ModosithatoResultSet:KnowledgeUnit { externalId: "ModosithatoResultSet", 
  name: "Módosítható resultSet", 
  description: "ResultSet-en keresztüli adat módosítás", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(EscapeSzintaxis:KnowledgeUnit { externalId: "EscapeSzintaxis", 
  name: "Escape szintaxis", 
  description: "Escape szekvenciák használata DB függetlenséghez", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),

(BlobKezeles:KnowledgeUnit { externalId: "BlobKezeles", 
  name: "BLOB kezelés", 
  description: "Bináris fájlok (képek, hangok) kezelése", 
  timeInstances: ["v1"], 
  evokers: [],
  topic: "JDBC_API_1"}),


(SqlApiJellemzese)-[:prerequisite_of]->(JdbcAlapfogalmak),
(JdbcAlapfogalmak)-[:prerequisite_of]->(JdbcDriverKezeles),
(JdbcAlapfogalmak)-[:prerequisite_of]->(JdbcOsztalyok),
(JdbcDriverKezeles)-[:prerequisite_of]->(KapcsolatKiepitese),
(JdbcOsztalyok)-[:prerequisite_of]->(KapcsolatKiepitese),
(KapcsolatKiepitese)-[:prerequisite_of]->(DmlParancsok),
(KapcsolatKiepitese)-[:prerequisite_of]->(DqlParancsok),
(KapcsolatKiepitese)-[:prerequisite_of]->(Hibakezeles),
(DmlParancsok)-[:prerequisite_of]->(PreparedStatementHasznalata),
(TranzakcioKezeles)-[:prerequisite_of]->(PreparedStatementHasznalata),
(DmlParancsok)-[:prerequisite_of]->(TranzakcioKezeles),
(Hibakezeles)-[:prerequisite_of]->(TranzakcioKezeles),
(DqlParancsok)-[:prerequisite_of]->(MetaadatKezeles),
(KapcsolatKiepitese)-[:prerequisite_of]->(CallableStatementHasznalata),
(PreparedStatementHasznalata)-[:prerequisite_of]->(CallableStatementHasznalata),
(TranzakcioKezeles)-[:prerequisite_of]->(CallableStatementHasznalata),
(DqlParancsok)-[:prerequisite_of]->(ModosithatoResultSet),
(DqlParancsok)-[:prerequisite_of]->(EscapeSzintaxis),
(PreparedStatementHasznalata)-[:prerequisite_of]->(BlobKezeles),


(TananyagSqlApiJellemzese:MaterialUnit { externalId: 3000,
description: "Tananyag",
url: "SqlApiJellemzese.pdf",
timeInstances: []}),

(TananyagJdbcAlapfogalmak:MaterialUnit { externalId: 3001,
description: "Tananyag",
url: "JdbcAlapfogalmak.pdf",
timeInstances: []}),

(TananyagJdbcDriverKezeles:MaterialUnit { externalId: 3002,
description: "Tananyag",
url: "JdbcDriverKezeles.pdf",
timeInstances: []}),

(TananyagJdbcOsztalyok:MaterialUnit { externalId: 3003,
description: "Tananyag",
url: "JdbcOsztalyok.pdf",
timeInstances: []}),

(TananyagKapcsolatKiepitese:MaterialUnit { externalId: 3004,
description: "Tananyag",
url: "KapcsolatKiepitese.pdf",
timeInstances: []}),

(TananyagDmlParancsok:MaterialUnit { externalId: 3005,
description: "Tananyag",
url: "DmlParancsok.pdf",
timeInstances: []}),

(TananyagDqlParancsok:MaterialUnit { externalId: 3006,
description: "Tananyag",
url: "DqlParancsok.pdf",
timeInstances: []}),

(TananyagHibakezeles:MaterialUnit { externalId: 3007,
description: "Tananyag",
url: "Hibakezeles.pdf",
timeInstances: []}),

(TananyagPreparedStatementHasznalata:MaterialUnit { externalId: 3008,
description: "Tananyag",
url: "PreparedStatementHasznalata.pdf",
timeInstances: []}),

(TananyagTranzakcioKezeles:MaterialUnit { externalId: 3009,
description: "Tananyag",
url: "TranzakcioKezeles.pdf",
timeInstances: []}),

(TananyagMetaadatKezeles:MaterialUnit { externalId: 3010,
description: "Tananyag",
url: "MetaadatKezeles.pdf",
timeInstances: []}),

(TananyagCallableStatementHasznalata:MaterialUnit { externalId: 3011,
description: "Tananyag",
url: "CallableStatementHasznalata.pdf",
timeInstances: []}),

(TananyagModosithatoResultSet:MaterialUnit { externalId: 3012,
description: "Tananyag",
url: "ModosithatoResultSet.pdf",
timeInstances: []}),

(TananyagEscapeSzintaxis:MaterialUnit { externalId: 3013,
description: "Tananyag",
url: "EscapeSzintaxis.pdf",
timeInstances: []}),

(TananyagBlobKezeles:MaterialUnit { externalId: 3014,
description: "Tananyag",
url: "BlobKezeles.pdf",
timeInstances: []}),


(TananyagSqlApiJellemzese)-[:material_of]->(SqlApiJellemzese),
(TananyagJdbcAlapfogalmak)-[:material_of]->(JdbcAlapfogalmak),
(TananyagJdbcDriverKezeles)-[:material_of]->(JdbcDriverKezeles),
(TananyagJdbcOsztalyok)-[:material_of]->(JdbcOsztalyok),
(TananyagKapcsolatKiepitese)-[:material_of]->(KapcsolatKiepitese),
(TananyagDmlParancsok)-[:material_of]->(DmlParancsok),
(TananyagDqlParancsok)-[:material_of]->(DqlParancsok),
(TananyagHibakezeles)-[:material_of]->(Hibakezeles),
(TananyagPreparedStatementHasznalata)-[:material_of]->(PreparedStatementHasznalata),
(TananyagTranzakcioKezeles)-[:material_of]->(TranzakcioKezeles),
(TananyagMetaadatKezeles)-[:material_of]->(MetaadatKezeles),
(TananyagCallableStatementHasznalata)-[:material_of]->(CallableStatementHasznalata),
(TananyagModosithatoResultSet)-[:material_of]->(ModosithatoResultSet),
(TananyagEscapeSzintaxis)-[:material_of]->(EscapeSzintaxis),
(TananyagBlobKezeles)-[:material_of]->(BlobKezeles),


(TesztSqlApiJellemzese:TestUnit { externalId: 300,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztJdbcAlapfogalmak:TestUnit { externalId: 301,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztJdbcDriverKezeles:TestUnit { externalId: 302,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztJdbcOsztalyok:TestUnit { externalId: 303,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztKapcsolatKiepitese:TestUnit { externalId: 304,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztDmlParancsok:TestUnit { externalId: 305,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztDqlParancsok:TestUnit { externalId: 306,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztHibakezeles:TestUnit { externalId: 307,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztPreparedStatementHasznalata:TestUnit { externalId: 308,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztTranzakcioKezeles:TestUnit { externalId: 309,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztMetaadatKezeles:TestUnit { externalId: 310,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztCallableStatementHasznalata:TestUnit { externalId: 311,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztModosithatoResultSet:TestUnit { externalId: 312,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztEscapeSzintaxis:TestUnit { externalId: 313,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztBlobKezeles:TestUnit { externalId: 314,
description: "Feleletválasztós teszt",
url: "multiple-choice",
noOfAskedQuestions: 5,
timeInstances: []}),

(TesztSqlApiJellemzese)-[:tests]->(SqlApiJellemzese),
(TesztJdbcAlapfogalmak)-[:tests]->(JdbcAlapfogalmak),
(TesztJdbcDriverKezeles)-[:tests]->(JdbcDriverKezeles),
(TesztJdbcOsztalyok)-[:tests]->(JdbcOsztalyok),
(TesztKapcsolatKiepitese)-[:tests]->(KapcsolatKiepitese),
(TesztDmlParancsok)-[:tests]->(DmlParancsok),
(TesztDqlParancsok)-[:tests]->(DqlParancsok),
(TesztHibakezeles)-[:tests]->(Hibakezeles),
(TesztPreparedStatementHasznalata)-[:tests]->(PreparedStatementHasznalata),
(TesztTranzakcioKezeles)-[:tests]->(TranzakcioKezeles),
(TesztMetaadatKezeles)-[:tests]->(MetaadatKezeles),
(TesztCallableStatementHasznalata)-[:tests]->(CallableStatementHasznalata),
(TesztModosithatoResultSet)-[:tests]->(ModosithatoResultSet),
(TesztEscapeSzintaxis)-[:tests]->(EscapeSzintaxis),
(TesztBlobKezeles)-[:tests]->(BlobKezeles)

"""
conn.query(cqlCreate5)

[]